# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify, plaase fallow this order:
- Title for each plot is mandatory
- Analysis must be written 
- Address of data should be mentioned

Loading in all our data

In [79]:
import yaml
import pandas as pd
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

# Figures



### **Comparison of Dehydration and control of one person**

### Explanations: 

### Data: 

In [ ]:
def clean_digit_span(raw_df):
    # Select the sequence length data from the raw data and create a dataframe
    seq_length_df = raw_df[raw_df[1].astype(str).str.match(r'\d+')]
    seq_length_df

    # Get the value of the longest sequence remebered
    longest = seq_length_df[2]
    longest = longest.tolist()

    # Get the number of errors made
    error_number = seq_length_df[3]
    error_number = error_number.tolist()

    click_stim_df = raw_df[raw_df[1]=='clickedStim']
    click_stim_df.size

    clicks_observed = click_stim_df.count(axis=1) - 2 
    clicks_observed = clicks_observed.tolist()

    clicks_expected =  pd.to_numeric(longest) + 1
    clicks_expected = clicks_expected.tolist()

    clean_data = pd.DataFrame(data ={'seq length':longest,
                        'errors': error_number,
                        'clicks expected': clicks_expected,
                        'clicks observed':clicks_observed})
    
    return clean_data


In [87]:
import re

def create_merged_df(config_dict, test_name):
     # selects only the files we want to use
    files = {name: file for name, file in config_dict.items() if re.search(test_name, name)} 
    # read the files 
    for test, file in files.items():
        df_dict = pd.read_excel(file, sheet_name=None, header=None)
        for session, df in df_dict.items():
            # extracting the participant name, type name and repeat number
            participant = test.split('_')[-1]
            type, repeat = session.split('_')
            # inserting the repeat, type, participant columns
            df.insert(0, 'repeat', repeat)
            df.insert(0, 'type', type)
            df.insert(0, 'participant', participant)
            # concatenate if df_all exists
            try:
                df_all = pd.concat([df_all, df])
            except UnboundLocalError:
                df_all = df
    
    return df_all

In [88]:
df_verbal_fluency = create_merged_df(config, 'verbal')
df_flanker = create_merged_df(config, 'flanker')
df_digit_span = create_merged_df(config, 'digit_span')
df_stroop = create_merged_df(config, 'stroop')
df_stop_signal = create_merged_df(config, 'stop')

### Analysis: 

---

### **Comparison of Dehydration of all participants**

### Explanations: 

#### Data: 

In [83]:
#code

### Analysis: 

---

### **Comparison of Rehydration of all participants**

### Explanations: 

### Data: 

In [84]:
#code

### Analysis: 

---